# Figure 5

## Imports

In [1]:
import dolfin
import matplotlib.pyplot as plt
import math
import numpy

import dolfin_mech                    as dmech
import micro_poro_structure_generator as gen

## Parameters

In [2]:
E_s  = 1
nu_s = 0.2

mu_s    = E_s/2/(1 + nu_s)
lmbda_s = E_s*nu_s/(1 + nu_s)/(1 - 2*nu_s)
k_s     = (3*lmbda_s + 2*mu_s)/3

mat_params = {"E":E_s, "nu":nu_s}

## Computing homogenized parameters for different geometries

### Voronoi inclusions

In [3]:
lmbda_lst_voronoi = []
mu_lst_voronoi    = []
phi_lst_voronoi   = []
kappa_lst_voronoi = []

mesh_filebasename = "Fig5-voronoi"
seeds_filename = "Fig5-voronoi-seeds.dat"

domain_y = 1.
domain_x = domain_y * numpy.sqrt(3)/1.5
shift_y = 0.

thickness_lst = [0.01, 0.04, 0.06, 0.09, 0.12, 0.15, 0.17]
for k_thickness, thickness in enumerate(thickness_lst):
    print(k_thickness)
    gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
        mesh_filename = mesh_filebasename,
        seeds_filename = seeds_filename,
        h = thickness,
        lcar = thickness/5,
        domain_x = domain_x,
        domain_y = domain_y,
        shift_y = shift_y,
        remove_seeds = False)

    mesh = dolfin.Mesh()
    dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)

    coord = mesh.coordinates()
    xmax = max(coord[:,0]); xmin = min(coord[:,0])
    ymax = max(coord[:,1]); ymin = min(coord[:,1])
    mesh_bbox = [xmin, xmax, ymin, ymax]

    V0 = (xmax - xmin)*(ymax - ymin)

    dV = dolfin.Measure(
        "dx",
        domain=mesh)
    Vs0 = dolfin.assemble(dolfin.Constant(1.) * dV)

    Vf0 = V0 - Vs0
    Phif0 = Vf0/V0

    homogenization_problem = dmech.HomogenizationProblem(
        dim = 2,
        mesh = mesh,
        mat_params = mat_params,
        vol = V0,
        bbox = mesh_bbox)
    [lambda_, mu_] = homogenization_problem.get_lambda_and_mu()
    kappa_ = homogenization_problem.get_kappa()

    phi_lst_voronoi.append(Phif0)
    lmbda_lst_voronoi.append(lambda_)
    mu_lst_voronoi.append(mu_)
    kappa_lst_voronoi.append(kappa_)

0
Info    : Clearing all models and views...
Info    : Done clearing all models and views


AttributeError: module 'scipy' has no attribute 'spatial'

In [ ]:
import pickle
open_file = open(seeds_filename, "rb")
points = pickle.load(open_file)
open_file.close()

In [ ]:
from scipy import spatial
scipy.spatial.Delaunay(points)

### Hexagonal inclusions

In [ ]:
lmbda_lst_hexagonal = []
mu_lst_hexagonal    = []
phi_lst_hexagonal   = []
kappa_lst_hexagonal = []

mesh_filebasename = "Fig5-hexagonal"
seeds_filename = "Fig5-hexagonal-seeds.dat"

domain_y = 1.
domain_x = domain_y * numpy.sqrt(3)/1.5/2
shift_y = 0.

gen.generate_seeds_semi_regular(
    DoI = 0.,
    row = 1,
    domain_y = domain_y,
    seeds_filename = seeds_filename)

for k_thickness in range(11):
    thickness = (k_thickness+0.5)/20

    gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
        mesh_filename = mesh_filebasename,
        seeds_filename = seeds_filename,
        h = thickness,
        lcar = thickness/5,
        domain_x = domain_x,
        domain_y = domain_y,
        shift_y = shift_y,
        remove_seeds = False)

    mesh = dolfin.Mesh()
    dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)

    coord = mesh.coordinates()
    xmax = max(coord[:,0]); xmin = min(coord[:,0])
    ymax = max(coord[:,1]); ymin = min(coord[:,1])
    mesh_bbox = [xmin, xmax, ymin, ymax]

    V0 = (xmax - xmin)*(ymax - ymin)

    dV = dolfin.Measure(
        "dx",
        domain=mesh)
    Vs0 = dolfin.assemble(dolfin.Constant(1.) * dV)

    Vf0 = V0 - Vs0
    Phif0 = Vf0/V0

    homogenization_problem = dmech.HomogenizationProblem(
        dim = 2,
        mesh = mesh,
        mat_params = mat_params,
        vol = V0,
        bbox = mesh_bbox)
    [lambda_, mu_] = homogenization_problem.get_lambda_and_mu()
    kappa_ = homogenization_problem.get_kappa()

    phi_lst_hexagonal.append(Phif0)
    lmbda_lst_hexagonal.append(lambda_)
    mu_lst_hexagonal.append(mu_)
    kappa_lst_hexagonal.append(kappa_)

[[0.28867513 0.25      ]
 [0.         0.75      ]
 [0.8660254  0.25      ]
 [0.57735027 0.75      ]]
Info    : Clearing all models and views...
Info    : Done clearing all models and views


AttributeError: module 'scipy' has no attribute 'spatial'

### Circular inclusions

In [ ]:
lmbda_lst_circular = []
mu_lst_circular    = []
phi_lst_circular   = []
kappa_lst_circular = []

mesh_filebasename = "Fig5-circular"

width = 1.
V0 = math.sqrt(3) * width**2

shift_x = 0.
shift_y = 0.

for Phif0 in range(1, 85, 2):
    Phif0 = Phif0/100
    r = math.sqrt(Phif0*V0/math.pi/2)

    gen.generate_mesh_2D_rectangle_w_circular_inclusions(
        mesh_filename = mesh_filebasename,
        width = width,
        r = r,
        lcar = width/40,
        shift_x = shift_x,
        shift_y = shift_y)
    
    mesh = dolfin.Mesh()
    dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)

    coord = mesh.coordinates()
    xmax = max(coord[:,0]); xmin = min(coord[:,0])
    ymax = max(coord[:,1]); ymin = min(coord[:,1])
    mesh_bbox = [xmin, xmax, ymin, ymax]

    V0 = (xmax - xmin)*(ymax - ymin)

    dV = dolfin.Measure(
        "dx",
        domain=mesh)
    Vs0 = dolfin.assemble(dolfin.Constant(1.) * dV)

    Vf0 = V0 - Vs0
    Phif0 = Vf0/V0

    homogenization_problem = dmech.HomogenizationProblem(
        dim = 2,
        mesh = mesh,
        mat_params = mat_params,
        vol = V0,
        bbox = mesh_bbox)
    [lambda_, mu_] = homogenization_problem.get_lambda_and_mu()
    kappa_ = homogenization_problem.get_kappa()

    phi_lst_circular.append(Phif0)
    lmbda_lst_circular.append(lambda_)
    mu_lst_circular.append(mu_)
    kappa_lst_circular.append(kappa_)


Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Ellipse)
Info    : [ 40%] Meshing curve 4 (Line)
Info    : [ 50%] Meshing curve 5 (Ellipse)
Info    : [ 60%] Meshing curve 6 (Line)
Info    : [ 70%] Meshing curve 7 (Ellipse)
Info    : [ 80%] Meshing curve 8 (Line)
Info    : [ 90%] Meshing curve 9 (Ellipse)
Info    : Done meshing 1D (Wall 0.00182273s, CPU 0.001932s)
Info    : Meshing 2D...
Info    : Meshing surface 8 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.124505s, CPU 0.121839s)
Info    : 3525 nodes 7059 elements
Info    : Writing 'Fig5-circular.msh'...
Info    : Done writing 'Fig5-circular.msh'
Info    : Running '/Users/tayebi/miniconda3/envs/all/bin/gmsh -2 -o Fig5-circular.msh -format msh22 Fig5-circular.msh' [Gmsh 4.9.0, 1 node, max. 1 thread]
Info    : Started on Thu Nov 23 14:24:31 20

## Post processing

### Adding limit porosities

In [ ]:
k_lst_voronoi   = []
k_lst_hexagonal = []
k_lst_circular  = []

phi_lst_voronoi.insert(0, 1.)
mu_lst_voronoi.insert(0, 0.)
lmbda_lst_voronoi.insert(0, 0.)
kappa_lst_voronoi.insert(0, 0.)

phi_lst_voronoi.append(0.)
mu_lst_voronoi.append(mu_s)
lmbda_lst_voronoi.append(lmbda_s)
kappa_lst_voronoi.append(k_s)

for i in range(len(phi_lst_voronoi)):
    k_lst_voronoi.append((3*lmbda_lst_voronoi[i] + 2*mu_lst_voronoi[i])/3)

phi_lst_hexagonal.insert(0, 1.)
mu_lst_hexagonal.insert(0, 0.)
lmbda_lst_hexagonal.insert(0, 0.)
kappa_lst_hexagonal.insert(0, 0.)

phi_lst_hexagonal.append(0.)
mu_lst_hexagonal.append(mu_s)
lmbda_lst_hexagonal.append(lmbda_s)
kappa_lst_hexagonal.append(k_s)

for i in range (len(phi_lst_hexagonal)):
    k_lst_hexagonal.append((3*lmbda_lst_hexagonal[i] + 2*mu_lst_hexagonal[i])/3)

phi_lst_circular.insert(0, 0.)
mu_lst_circular.insert(0, mu_s)
lmbda_lst_circular.insert(0, lmbda_s)
kappa_lst_circular.insert(0, k_s)

phi_lst_circular.append(1.)
mu_lst_circular.append(0.)
lmbda_lst_circular.append(0.)
kappa_lst_circular.append(0.)

for i in range (len(phi_lst_circular)):
    k_lst_circular.append((3*lmbda_lst_circular[i] + 2*mu_lst_circular[i])/3)

### Computing homogenized parameters for the dilute and differential schemes [Dormieux, 2006]

In [ ]:
def Eq1(mu_hom, phi):
    return ((1+4*mu_s/(3*k_s))*(mu_hom/mu_s)**3)/(2 - (1 - 4*mu_s/(3*k_s))*(mu_hom/mu_s)**(3/5)) - (1 - phi)**6

def Eq2(mu_hom, k_hom):
    return mu_hom/mu_s - ((-4/3*mu_hom/k_hom + 1)**(5/3))/((-4/3*mu_s/k_s + 1)**(5/3))

def differential_scheme(phi_0):
    h = 10e-6
    mu_hom = mu_s
    k_hom = k_s
    res = 1
    while (res > 10e-7):
        res = Eq1(mu_hom, phi_0)
        jac = (Eq1(mu_hom + h, phi_0) - Eq1(mu_hom, phi_0))/h
        delta = - res/jac
        mu_hom = mu_hom + delta

    res = 1
    while (res > 10e-7):
        res = Eq2(mu_hom, k_hom)
        jac = (Eq2(mu_hom, k_hom + h) - Eq2(mu_hom, k_hom))/h
        delta = - res/jac
        k_hom = k_hom + delta

    return mu_hom, k_hom

def k_diff_func(mu_hom):
    return 4/3*mu_hom/(1 - (mu_hom/mu_s)**(3/5)*(1 - 4/3*mu_s/k_s))

In [ ]:
mu_diff = []
k_diff  = []
mu_dil  = []
k_dil   = []
for i in range(len(phi_lst_hexagonal)):
    mu_ = differential_scheme(phi_lst_hexagonal[i])[0]
    mu_diff.append(mu_)
    k_diff.append(k_diff_func(mu_))
    mu_dil.append(mu_s*(1 - 5*phi_lst_hexagonal[i]*(3*k_s + 4*mu_s)/(9*k_s + 8*mu_s)))
    k_dil.append(k_s*(1 - phi_lst_hexagonal[i]*(1 + 3*k_s/4/mu_s)))

### Normalizing homogenized parameters

In [ ]:
mu_lst_norm_voronoi    = []
k_lst_norm_voronoi     = []
kappa_lst_norm_voronoi = []

mu_lst_hexagonal_norm    = []
k_lst_hexagonal_norm     = []
kappa_lst_hexagonal_norm = []

mu_lst_norm_circular    = []
k_lst_norm_circular     = []
kappa_lst_norm_circular = []

mu_diff_norm = []
k_diff_norm  = []

mu_dil_norm = []
k_dil_norm  = []

for i in range (len(mu_lst_hexagonal)):
    mu_lst_hexagonal_norm.append(mu_lst_hexagonal[i]/mu_s)
    mu_dil_norm.append(mu_dil[i]/mu_s)
    mu_diff_norm.append(mu_diff[i]/mu_s)
    k_lst_hexagonal_norm.append(k_lst_hexagonal[i]/k_s)
    k_diff_norm.append(k_diff[i]/k_s)
    k_dil_norm.append(k_dil[i]/k_s)
    kappa_lst_hexagonal_norm.append(kappa_lst_hexagonal[i]/k_s)

for i in range (len(mu_lst_voronoi)):
    mu_lst_norm_voronoi.append(mu_lst_voronoi[i]/mu_s)
    k_lst_norm_voronoi.append(k_lst_voronoi[i]/k_s)
    kappa_lst_norm_voronoi.append(kappa_lst_voronoi[i]/k_s)

for i in range (len(mu_lst_circular)):
    mu_lst_norm_circular.append(mu_lst_circular[i]/mu_s)
    k_lst_norm_circular.append(k_lst_circular[i]/k_s)
    kappa_lst_norm_circular.append(kappa_lst_circular[i]/k_s)

### Generating plots

In [ ]:
phi_dil = numpy.linspace(min(phi_lst_hexagonal),(9*k_s + 8*mu_s)/(3*k_s + 4*mu_s)/5,100)
mu_dil_norm = 1 - 5*phi_dil*(3*k_s + 4*mu_s)/(9*k_s + 8*mu_s)

plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$\Phi_{f0}$', fontsize=16)
plt.ylabel(r'$\tilde{\mu}/\mu_s$', fontsize=16)

plt.plot(phi_lst_circular, mu_lst_norm_circular, '#084594')
plt.plot(phi_lst_hexagonal, mu_lst_hexagonal_norm, '#084594', linestyle='dashdot')
plt.plot(phi_lst_voronoi, mu_lst_norm_voronoi, '#084594', linestyle='dashed')
plt.plot(phi_lst_hexagonal, mu_diff_norm, '#005824')
plt.plot(phi_dil, mu_dil_norm, '#005824', linestyle='dashed')

plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend(['Microscopic model - circular inclusions', 'Microscopic model - hexagonal inclusions', 'Microscopic model - polygonal inclusions', 'Differential scheme [Dormieux et al. 2006]', 'Dilute scheme [Dormieux et al. 2006]'])

plt.text(phi_lst_hexagonal[9]-0.02, mu_lst_hexagonal_norm[9]-0.08, '(a)',fontsize=12)
plt.text(phi_lst_hexagonal[6]-0.015, mu_lst_hexagonal_norm[6]-0.07, '(b)',fontsize=12)
plt.text(phi_lst_hexagonal[4]-0.025, mu_lst_hexagonal_norm[4]-0.04, '(c)',fontsize=12)
plt.text(phi_lst_hexagonal[2]-0.015, mu_lst_hexagonal_norm[2]+0.02, '(d)',fontsize=12)

plt.show()
# plt.savefig('Fig5-mu.pdf', bbox_inches='tight')

In [ ]:
phi_dil = numpy.linspace(min(phi_lst_hexagonal),4*mu_s/(4*mu_s + 3*k_s),100)
k_dil_norm = (1 - phi_dil*(1 + 3*k_s/4/mu_s))

plt.figure()

plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$\Phi_{f0}$', fontsize = 16)
plt.ylabel(r'$\tilde{k}/k_s$', fontsize = 16)

plt.plot(phi_lst_circular, k_lst_norm_circular, '#084594')
plt.plot(phi_lst_hexagonal, k_lst_hexagonal_norm, '#084594', linestyle='dashdot')
plt.plot(phi_lst_voronoi, k_lst_norm_voronoi, '#084594', linestyle='dashed')
plt.plot(phi_lst_hexagonal, k_diff_norm, '#005824')
plt.plot(phi_dil, k_dil_norm, '#005824', linestyle='dashed')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend(['Microscopic model - circular inclusions', 'Microscopic model - hexagonal inclusions', 'Microscopic model - polygonal inclusions', 'Differential scheme [Dormieux et al. 2006]', 'Dilute scheme [Dormieux et al. 2006]'])

plt.text(phi_lst_hexagonal[9]+0.01, k_lst_hexagonal_norm[9], '(a)',fontsize=12)
plt.text(phi_lst_hexagonal[6], k_lst_hexagonal_norm[6]+0.025, '(b)',fontsize=12)
plt.text(phi_lst_hexagonal[4]-0.015, k_lst_hexagonal_norm[4]+0.025, '(c)',fontsize=12)
plt.text(phi_lst_hexagonal[2]-0.015, k_lst_hexagonal_norm[2]+0.03, '(d)',fontsize=12)

plt.show()
# plt.savefig('Fig5-k.pdf',bbox_inches='tight')